# Ada-DiffAct Model using Keras

## libraries to be used

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

%load_ext tensorboard
import datetime



## Custom Activation Functions

In [2]:
initializer0 = keras.initializers. RandomUniform(minval = -1, maxval =2)
initializer1 = keras.initializers. RandomUniform(minval = -1, maxval =2)
initializer2 = keras.initializers. RandomUniform(minval = -0.05, maxval =0.05)

class Diffact(keras.layers.Layer):
    def __init__(self):
        super(Diffact, self).__init__()
        self.k0 = self.add_weight(name='k0', shape = (), initializer=initializer0, trainable=True)
        self.k1 = self.add_weight(name='k1', shape = (), initializer=initializer1, trainable=True)
        self.k2 = self.add_weight(name='k2', shape = (), initializer=initializer2, trainable=True)
        
    def call(self, inputs):
        return self.k0 + tf.multiply(inputs, self.k1) + tf.multiply(tf.math.multiply(inputs,inputs), self.k2)

## Model Graph

training on MNIST, trial 1

In [3]:
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
inputs = keras.Input(shape=input_shape)
x = layers.Conv2D(32, kernel_size=(3, 3))(inputs)
diffact = Diffact()
x = diffact(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(64, kernel_size=(3, 3))(x)
x = diffact(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_diffact")
model.summary()

Model: "mnist_diffact"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 26, 26, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
diffact (Diffact)               multiple             3           conv2d[0][0]                     
                                                                 conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 13, 13, 32)   0           diffact[0][0]        

In [5]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

log_dir = "mnistlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[tensorboard_callback])

Epoch 1/15
  1/422 [..............................] - ETA: 0s - loss: 2.3464 - accuracy: 0.1484WARNING:tensorflow:From C:\Users\shamb\anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
422/422 [==============================] - 43s 103ms/step - loss: 0.3366 - accuracy: 0.8969 - val_loss: 0.0907 - val_accuracy: 0.9748
Epoch 2/15
422/422 [==============================] - 47s 111ms/step - loss: 0.1101 - accuracy: 0.9664 - val_loss: 0.0583 - val_accuracy: 0.9838
Epoch 3/15
422/422 [==============================] - 67s 160ms/step - loss: 0.0840 - accuracy: 0.9742 - val_loss: 0.0552 - val_accuracy: 0.9852
Epoch 4/15
422/422 [==============================] - 64s 151ms/step - loss: 0.0736 - accuracy: 0.9775 - val_loss: 0.0472 - val_accuracy: 0.9870
Epoch 5/15
422/422 [==============================] - 

In [6]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.02574027143418789
Test accuracy: 0.9911999702453613


In [7]:
print("AF coefficients (weights) {}".format(diffact.get_weights()))

AF coefficients (weights) [-0.13069697, 1.2613214, 0.7247993]


In [13]:
%tensorboard --logdir mnistlogs/fit --port=6051

Reusing TensorBoard on port 6051 (pid 4004), started 0:00:40 ago. (Use '!kill 4004' to kill it.)